In [49]:
from nba_api.live.nba.endpoints import boxscore
import pandas as pd

import psycopg2
from sqlalchemy import create_engine
import sqlconfig
from sqlalchemy.exc import SQLAlchemyError

import requests

In [ ]:
#Finding only valid games
def selectGames():
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )

        query = "SELECT g.game_id FROM nba24_25.nba_games g;"

        try:
            data=pd.read_sql(query, engine)
            return data
        
        except Exception as e:
            print(f"An error occurred: {e}")
        

        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
#Finding all player boxscores
def player_boxscores(game_ids):
    boxscores_final=[]
    for id in game_ids:
        try: 
            box = boxscore.BoxScore(id) 
            data=box.game.get_dict()
        except requests.exceptions.Timeout:
            print("Time out Error")
            data=[]
        
        #Home Team Data Wrangling
        home_players_boxscores = pd.json_normalize(data['homeTeam']['players'])
        if 'statistics' in home_players_boxscores.columns:
            data_stats = pd.json_normalize(home_players_boxscores['statistics'])
            home_players_boxscores = home_players_boxscores.drop(columns=['statistics']).join(data_stats) 
        home_players_boxscores['HomeOrAway']='Home'
        home_players_boxscores.columns = home_players_boxscores.columns.str.replace('statistics.', '')

        #Away Team Data Wrangling
        away_players_boxscores = pd.json_normalize(data['awayTeam']['players'])
        if 'statistics' in away_players_boxscores.columns:
            data_stats = pd.json_normalize(away_players_boxscores['statistics'])   
            away_players_boxscores = away_players_boxscores.drop(columns=['statistics']).join(data_stats)
        away_players_boxscores['HomeOrAway'] = 'Away'
        away_players_boxscores.columns = away_players_boxscores.columns.str.replace('statistics.', '')

        #Combining individual game boxscores
        boxscores=pd.concat([home_players_boxscores,away_players_boxscores],ignore_index=True)
        boxscores['GAME_ID']=id
        boxscores_final.append(boxscores)

    #Combining all boxscores
    boxscores_final = pd.concat(boxscores_final, ignore_index=True)
    boxscores_final=boxscores_final.drop(columns=['order','jerseyNum','oncourt','nameI','firstName','familyName','minutesCalculated','plus'])

    #Extracting columns into BCNF
    
    #Injury Report
    injury_report=boxscores_final[["GAME_ID","personId","notPlayingReason","notPlayingDescription"]]
    injury_report.columns=['game_id','player_id','not_playing_reason','not_playing_description']
    injury_report=injury_report.copy()
    injury_report['game_id']=injury_report['game_id'].astype(str)
    injury_report['player_id']=injury_report['player_id'].astype(str)
    print("Injury Report")
    print(injury_report.head(5))
    
    #Boxscore
    boxscores=boxscores_final[["GAME_ID","personId","starter","played","assists","blocks","blocksReceived","fieldGoalsAttempted","fieldGoalsMade","foulsOffensive","foulsDrawn","foulsPersonal","foulsTechnical","freeThrowsMade","freeThrowsAttempted",
                               "minutes","plusMinusPoints","pointsFastBreak","pointsInThePaint","pointsSecondChance","reboundsOffensive","reboundsDefensive","steals","threePointersMade","threePointersAttempted","turnovers","twoPointersMade","twoPointersAttempted",'HomeOrAway']]
    
    boxscores.columns=['game_id','player_id','starter','played','assists','blocks','blocks_received','fgm','fga','fouls_offensive','fouls_drawn','fouls_personal','fouls_technical','ftm','fta','minutes_played','plus_minus','points_fast_break','points_in_paint',
                       'points_second_chance','oreb','dreb','stl','fg3m','fg3a','turnovers','fg2m','fg2a','homeoraway']
    boxscores=boxscores.copy()
    boxscores['game_id']=boxscores['game_id'].astype(str)
    boxscores['player_id']=boxscores['player_id'].astype(str)
    
    print("Boxscore")
    print(boxscores.head(5))

    return boxscores,injury_report


In [ ]:
def intitalInsertion(data,tablename):
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )
        
        # Insertions into the tables
        data.to_sql(tablename, engine, schema="nba24_25", if_exists="append", index=False)
      
        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
#Finding Active players
def selectPlayers():
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )

        query = "SELECT p.player_id FROM nba24_25.active_nba_players p;"

        try:
            data=pd.read_sql(query, engine)
            return data
        
        except Exception as e:
            print(f"An error occurred: {e}")
        

        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
#Removing unactive players
def active_players(data,players):
    data=data[data['player_id'].isin(players['player_id'])]
    return data

In [55]:
game_ids=selectGames()
game_ids=game_ids['game_id'].unique()

In [56]:
boxscores,injury_report=player_boxscores(game_ids)

Injury Report
      game_id player_id not_playing_reason not_playing_description
0  0022400835   1629060                NaN                     NaN
1  0022400835      2544                NaN                     NaN
2  0022400835   1629637                NaN                     NaN
3  0022400835   1630559                NaN                     NaN
4  0022400835   1629029                NaN                     NaN
Boxscore
      game_id player_id starter played  assists  blocks  blocks_received  fga  \
0  0022400835   1629060       1      1        0       2                1   12   
1  0022400835      2544       1      1        3       0                1   17   
2  0022400835   1629637       1      1        0       2                1    5   
3  0022400835   1630559       1      1        5       0                1   12   
4  0022400835   1629029       1      1       12       2                3   17   

   fgm  fouls_offensive  ...  points_second_chance  oreb  dreb  stl  fg3m  \
0    6     

In [57]:
players=selectPlayers()


In [58]:
boxscores=active_players(boxscores,players)
injury_report=active_players(injury_report,players)

In [ ]:
intitalInsertion(boxscores,"boxscores")
intitalInsertion(injury_report,"injury_report")

Data inserted
